In [1]:
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import load_iris

import streamlit as st
import pickle 

In [2]:
data = load_iris()

# Predictor columns
X = data.data

# Target column
y = data.target

# Create a dataframe
df = pd.DataFrame(X, columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']) 
df['Species'] = data.target

# Print dataframe
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 5.4 KB


In [4]:
# Replaces target class codes with text
df['Species'] = df['Species'].replace([0, 1, 2], ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
# Create predictor and target variables
X = df.drop('Species', axis=1)
y = df['Species']

# x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [7]:
# Create a pipeline for scaling and logistic regression model
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression())
])

# Fit the pipeline on the dataset
pipe.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LogisticRegression())])

In [8]:
# Test the pipeline using data from the same dataset
example = pd.DataFrame([[5.1, 3.5, 1.4, 0.2]])
example.columns = X.columns
pipe.predict(example)[0]

'Iris-setosa'

In [10]:
pipe.predict(example)

array(['Iris-setosa'], dtype=object)

In [9]:
# Pickle the pipeline
import joblib

with open("model.pkl", "wb") as f:
    joblib.dump(pipe, f)

In [36]:
# Create a function for Google Cloud Function
# All of this code would need to be written in a file called main.py:

import json
#​
def iris_prediction(sepal_length, 
                    sepal_width, 
                    petal_length, 
                    petal_width,
                    features):
    """
    Given sepal length, sepal width, petal length, and petal width,
    predict the class of iris
    """
    
    # Load the model from the file
    with open("model.pkl", "rb") as f:
        model = joblib.load(f)
        
    # Construct the 2D matrix of values that .predict is expecting
    X = [[sepal_length, sepal_width, petal_length, petal_width]]
    
    data = pd.DataFrame(X)
    data.columns = features
    
    # Get a list of predictions and select only 1st
    predictions = model.predict(data)
    prediction = predictions[0]
    
    return {"predicted_class": prediction}
#​
def predict(request):
    """
    `request` is an HTTP request object that will automatically be passed
    in by Google Cloud Functions
    
    You can find all of its properties and methods here:
    https://flask.palletsprojects.com/en/1.0.x/api/#flask.Request
    """
    # Get the request data from the user in JSON format
    request_json = request.get_json()
    
    # We are expecting the request to look like this:
    # {"sepal_length": <x1>, "sepal_width": <x2>, "petal_length": <x3>, "petal_width": <x4>}
    # Send it to our prediction function using ** to unpack the arguments
    result = iris_prediction(**request_json)
    
    # Return the result as a string with JSON format
    return json.dumps(result)

In [2]:
# import streamlit as st
# import pickle 
# import pandas as pd


with open('model.pkl','rb') as file:
    model_new = pickle.load(file)


s_p_l = st.number_input('Sepal Length')
s_p_w = st.number_input('Sepal Width')
p_t_l = st.number_input('Petal Length')
p_t_w = st.number_input('Petal Width')

if st.button("predict"):
    data = {
        "SepalLengthCm":[s_p_l],
        "SepalWidthCm":[s_p_w],
        "PetalLengthCm":[p_t_l],
        "PetalWidthCm":[p_t_w]  
    }

    df = pd.DataFrame(data)

    pred = model_new.predict(df)
    """
    ## Inputs
    """
    df

    """
    ## Predicted Class
    """

    pred

2023-11-28 13:57:53.074 
  command:

    streamlit run C:\Users\USER PC\anaconda3\envs\stream-env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [3]:
import streamlit as st

st.write("""
Below are our sales predictions for this customer""")

In [10]:
import pandas as pd

st.write("Here's our first attempt at using data to create a table:")
st.write(pd.DataFrame({
    'first column': [1, 2, 3, 4],
    'second column': [10, 20, 30, 40]
}))